Варіант No4
Розробити програму, яка працює із журналом реєстації автомобілів на
підприємстві (скористатись колекцією). Автомобіль може мати лише одного
користувача. Відповідно в класі повинна міститись інформація про поточну
особу, яка володіє правом користання автомобілем на даний момент. При зміні
користувача, інформація про попереднього користувача автомобіля повинна
зберігатись в системі. При написанні програми скористатись шаблоном
Immutable (Незмінний)

In [4]:
from dataclasses import dataclass, field
from typing import List, Optional

@dataclass(frozen=True)
class User:
    name: str
    position: str

@dataclass(frozen=True)
class CarHistory:
    user: User
    start_date: str
    end_date: Optional[str] = None  # None означає, що користувач ще використовує авто

@dataclass(frozen=True)
class Car:
    license_plate: str
    current_user: User
    history: List[CarHistory] = field(default_factory=list)

    def assign_new_user(self, new_user: User, start_date: str) -> "Car":
        # Створюємо оновлену історію
        updated_history = self.history.copy()

        # Оновлюємо кінець періоду користування для попереднього користувача
        if updated_history:
            last_history_entry = updated_history[-1]
            updated_history[-1] = CarHistory(
                user=last_history_entry.user,
                start_date=last_history_entry.start_date,
                end_date=start_date
            )

        # Додаємо нового користувача до історії
        updated_history.append(CarHistory(user=new_user, start_date=start_date))

        # Повертаємо новий об'єкт Car з оновленим користувачем і історією
        return Car(license_plate=self.license_plate, current_user=new_user, history=updated_history)

    def get_history(self) -> List[CarHistory]:
        return self.history

    def get_current_info(self) -> str:
        return f"Поточний користувач: {self.current_user.name}, Посада: {self.current_user.position}, Номер авто: {self.license_plate}"

# Демонстрація інтерактивної програми
def main():
    # Створюємо початкового користувача і автомобіль
    initial_user = User(name="Іван Іванов", position="Менеджер")
    car = Car(license_plate="AA1234BB", current_user=initial_user)
    car = car.assign_new_user(initial_user, "2024-01-01")

    while True:
        print("\nВиберіть опцію:")
        print("1. Змінити користувача автомобіля")
        print("2. Показати поточну інформацію про автомобіль")
        print("3. Показати історію користувачів")
        print("4. Вихід")

        choice = input("Ваш вибір: ")

        if choice == "1":
            name = input("Введіть ім'я нового користувача: ")
            position = input("Введіть посаду нового користувача: ")
            start_date = input("Введіть дату початку користування (формат: YYYY-MM-DD): ")
            new_user = User(name=name, position=position)
            car = car.assign_new_user(new_user, start_date)
            print(f"Користувач {name} був призначений на автомобіль з {start_date}.")

        elif choice == "2":
            print(car.get_current_info())

        elif choice == "3":
            print("Історія користувачів автомобіля:")
            for entry in car.get_history():
                end_date = entry.end_date if entry.end_date else "Ще користується"
                print(f"Користувач: {entry.user.name}, Посада: {entry.user.position}, Початок: {entry.start_date}, Кінець: {end_date}")

        elif choice == "4":
            print("Вихід із програми.")
            break

        else:
            print("Неправильний вибір. Спробуйте ще раз.")

# Запуск програми
if __name__ == "__main__":
    main()




Виберіть опцію:
1. Змінити користувача автомобіля
2. Показати поточну інформацію про автомобіль
3. Показати історію користувачів
4. Вихід


KeyboardInterrupt: Interrupted by user

Варіант No2
Розробити програму, яка працює зі списком книг. Розробити власний клас
колекцію, який містить усі необхідні методи (додавання, пошук, видалення).
Пошук книги здійснювати за автором. Доступ до елементів колекції здійснити
за допомогою шаблону Proxy (Заступник).

In [6]:
# Клас для представлення книги
class Book:
    def __init__(self, author, title):
        self.author = author
        self.title = title

    def __str__(self):
        return f"'{self.title}' by {self.author}"


# Клас для роботи з колекцією книг
class BookCollection:
    def __init__(self):
        self.books = []

    # Метод для додавання книги
    def add_book(self, book):
        self.books.append(book)
        print(f"Книга {book} додана до колекції.")

    # Метод для видалення книги
    def remove_book(self, title):
        for book in self.books:
            if book.title == title:
                self.books.remove(book)
                print(f"Книга '{title}' видалена з колекції.")
                return
        print(f"Книга '{title}' не знайдена в колекції.")

    # Метод для пошуку книг за автором
    def search_by_author(self, author):
        found_books = [book for book in self.books if book.author == author]
        return found_books


# Клас-заступник (Proxy) для колекції книг
class BookCollectionProxy:
    def __init__(self):
        self.collection = BookCollection()

    # Метод для додавання книги через Proxy
    def add_book(self, book):
        print("Proxy: Додавання книги до колекції...")
        self.collection.add_book(book)

    # Метод для видалення книги через Proxy
    def remove_book(self, title):
        print("Proxy: Видалення книги з колекції...")
        self.collection.remove_book(title)

    # Метод для пошуку книг через Proxy
    def search_by_author(self, author):
        print(f"Proxy: Пошук книг автора '{author}'...")
        found_books = self.collection.search_by_author(author)
        if found_books:
            print(f"Знайдено книги автора {author}:")
            for book in found_books:
                print(f"- {book}")
        else:
            print(f"Книг автора {author} не знайдено.")


# Функція для взаємодії з користувачем через консоль
def main():
    proxy = BookCollectionProxy()

    while True:
        print("\nМеню:")
        print("1. Додати книгу")
        print("2. Видалити книгу")
        print("3. Пошук книги за автором")
        print("4. Вийти")
        choice = input("Оберіть дію (1-4): ")

        if choice == "1":
            author = input("Введіть автора книги: ")
            title = input("Введіть назву книги: ")
            book = Book(author, title)
            proxy.add_book(book)

        elif choice == "2":
            title = input("Введіть назву книги, яку потрібно видалити: ")
            proxy.remove_book(title)

        elif choice == "3":
            author = input("Введіть автора для пошуку книг: ")
            proxy.search_by_author(author)

        elif choice == "4":
            print("Вихід з програми.")
            break

        else:
            print("Невірний вибір, спробуйте ще раз.")


if __name__ == "__main__":
    main()



Меню:
1. Додати книгу
2. Видалити книгу
3. Пошук книги за автором
4. Вийти
Оберіть дію (1-4): 1


KeyboardInterrupt: Interrupted by user